ToDO:

1.make def json_bydate(dateval) faster
2. split into two notebooks



3.add to change_on color bar



In [1]:
%matplotlib inline


import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import world_bank_data as wb

import json


from datetime import datetime, date



In [2]:

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Load the world map

In [3]:
# %%writeandexecute -i identifier load.py

def loadGeopandas():    
    shapefile = 'data/countries_110m/ne_110m_admin_0_countries.shp'
    gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
    gdf.head()
    gdf.columns = ['country', 'country_code', 'geometry']
    gdf.head()
    
    print(gdf[gdf['country'] == 'Antarctica'])
    #Drop row corresponding to 'Antarctica'
    gdf = gdf.drop(gdf.index[159])
    return gdf

In [4]:
gdf = loadGeopandas()
gdf.head()

        country country_code  \
159  Antarctica          ATA   

                                              geometry  
159  MULTIPOLYGON (((-48.66062 -78.04702, -48.15140...  


,country,country_code,geometry
0,Fiji,FJI,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,United Republic of Tanzania,TZA,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,Western Sahara,SAH,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Canada,CAN,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,United States of America,USA,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


# Get data from CSSEGISandData

In [5]:
def getTimeSeries(fname):
    print(fname)
    df = pd.read_csv(fname)
    if not fname.count('web-data'):
        countryData = df.groupby('Country/Region').sum()
        return countryData.drop([ 'Lat', 'Long'], axis=1)
    return df
    
def fetchCountryData(freq='1d'):
    gisandata ='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_'
    confirmed = getTimeSeries(f'{gisandata}confirmed_global.csv')

    return confirmed
   

In [21]:
country_df = getTimeSeries('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')
country_df.columns = map(str.lower, country_df.columns)



https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv


$\textbf{Show first entries the most up-to-date data table}$

# Show the most affected countries

In [7]:
def highlight_col(x):
    r = 'background-color: red'
    p = 'background-color: purple'
    g = 'background-color: grey'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = p
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    return df1


def show_latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='15')

interactive(children=(Text(value='15', description='n'), Output()), _dom_classes=('widget-interact',))

<function __main__.show_latest_cases(n)>

$\textbf{Merge the covid data with the world map}$

In [8]:
# Get csv with the countries and alpha codes
countries_code = pd.read_csv('JohnsHopkins-to-A3.csv')
countries_code.head()

# ToDO: merge to country_df according to country field
countries_code = countries_code.rename(columns={'Country/Region': 'country_region'})
countries_code = countries_code.rename(columns={'alpha3': 'country_code'})

country_df_code = pd.merge(countries_code, country_df, on='country_region')
country_df_code.head()

,country_region,country_code,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
0,US,USA,2020-04-19 10:30:50,40.00000,-100.000000,735287,39090,66819,629378,223.175189,NaN,NaN,5.316291,840,USA
1,Afghanistan,AFG,2020-04-19 10:30:30,33.93911,67.709953,993,32,131,830,2.550841,NaN,NaN,3.222558,4,AFG
2,Albania,ALB,2020-04-19 10:30:30,41.15330,20.168300,562,26,314,222,19.528807,NaN,NaN,4.626335,8,ALB
3,Algeria,DZA,2020-04-19 10:30:30,28.03390,1.659600,2534,367,894,1273,5.778654,NaN,NaN,14.483031,12,DZA
4,Andorra,AND,2020-04-19 10:30:30,42.50630,1.521800,704,35,205,464,911.149939,NaN,NaN,4.971591,20,AND


In [9]:
merged = gdf.merge(country_df_code, on = 'country_code')
merged.head()
#Replace NaN values to string 'No data'.
# merged.fillna(0, inplace = True)

,country,country_code,geometry,country_region,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
0,Fiji,FJI,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",Fiji,2020-04-19 10:30:30,-17.713400,178.065000,17,0,0,17,1.896382,NaN,NaN,0.000000,242,FJI
1,United Republic of Tanzania,TZA,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",Tanzania,2020-04-19 10:30:30,-6.369028,34.888822,147,5,11,131,0.246090,NaN,NaN,3.401361,834,TZA
2,Canada,CAN,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",Canada,2020-04-19 10:38:15,60.001000,-95.001000,34386,1520,11216,21650,90.834401,NaN,NaN,4.420404,124,CAN
3,United States of America,USA,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",US,2020-04-19 10:30:50,40.000000,-100.000000,735287,39090,66819,629378,223.175189,NaN,NaN,5.316291,840,USA
4,Kazakhstan,KAZ,"POLYGON ((87.35997 49.21498, 86.59878 48.54918...",Kazakhstan,2020-04-19 10:30:30,48.019600,66.923700,1661,17,382,1262,8.846067,NaN,NaN,1.023480,398,KAZ


In [10]:
def get_strdate(val):
    mounth = val.strftime("%m")
    day = val.strftime("%d")
    yer = val.strftime("%y")
    
    if mounth[0] == '0':
        mounth = mounth[1]

    if day[0] == '0':
        day = day[1]
    return '/'.join([mounth,day, yer])

In [26]:
data = fetchCountryData()
data = data.rename(columns={'Country/Region': 'country_region'})

https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv


In [34]:
def mergegdf_bydate(dateval):
    # dateval as str

    print(data.head())
    for col in data.columns: 
        print(col) 
    tmp_df = data[[r'Country/Region', dateval]].copy()
    tmp_df = tmp_df.rename(columns={dateval: 'confirmed'})
    
    max_conf_bydate =max( tmp_df['confirmed'])

    tmp_df_code = pd.merge(countries_code, tmp_df, on='country_region')
    
    merged = gdf.merge(tmp_df_code, on = 'country_code', how = 'left')
#     merged.fillna(0, inplace = True)
    return merged, max_conf_bydate, tmp_df_code

In [12]:
max_confirmed =max( country_df_code['confirmed'])
print(max_confirmed)

735287


In [13]:
import matplotlib.colors as colors
from datetime import timedelta, date, datetime
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

def create_imgs(single_date, fname):
    merged, maxconf, tmp_df = mergegdf_bydate(get_strdate(single_date))
    
    fig, ax = plt.subplots(1,1, figsize=(20, 12))
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2.5%", pad=0.01)
    print(merged[merged['country_region']=='Australia'])
#     missing values
#     merged.loc[np.random.choice(merged.index, 40), 'confirmed'] = np.nan

#     merged.plot(column='confirmed');
    merged.plot(column = 'confirmed', colormap = 'inferno_r',
                     linewidth=0.8, ax=ax, edgecolor='0.8', 
                     legend = True, 
                     legend_kwds={'label': "Number of confirmed cases"},
                     norm=colors.LogNorm(vmin=1, vmax=max_confirmed), 
                    cax = cax)
    
    ax.axis('off')
    ax.text(0.3, 1.1, 'Confirmed cases on {}'.format(single_date), 
                 fontdict =  {'fontsize': 25, 'fontweight' : '3'}, 
                transform = ax.transAxes)

    '''    ax.text(1.4, 0.4, 'Number of confirmed cases',
                 fontdict =  {'fontsize': 20, 'fontweight' : '3'}, 
                transform = ax.transAxes, rotation=-90)
    '''
    ax.annotate('Source: CSSEGISandData',xy=(0.1, .1),  
            xycoords='figure fraction', horizontalalignment='left',
            verticalalignment='top', fontsize=12, color='#555555')
    
    os.makedirs("output2", exist_ok=True)
    fig.savefig(fname)
    plt.close(fig)
    print(tmp_df.head())
    return tmp_df

In [14]:
def create_gif(start, end, images):
    exportname = 'map_{}_{}_2.gif'.format(start, end)
    kargs = { 'duration': 3 }
    imageio.mimsave(exportname, images[::1], 'GIF', **kargs)

    print('Checkout your new gif', exportname)


In [15]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)


In [35]:
import os
import numpy as np
import imageio

today  = datetime.date(datetime.now())
start = date(2020, 4, 5)
end = date(2020, 4, 6)

    
        
images = []
for single_date in daterange(start, end):
    fname = f'output2/{single_date.strftime("%d_%m_%Y")}.png'
    print(single_date)
    tmp_df = create_imgs(single_date, fname)

    images.append(imageio.imread(fname))
    print(tmp_df.head())

# create_gif(start, end, images)

2020-04-05
                1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  \
Country/Region                                                                  
Afghanistan           0        0        0        0        0        0        0   
Albania               0        0        0        0        0        0        0   
Algeria               0        0        0        0        0        0        0   
Andorra               0        0        0        0        0        0        0   
Angola                0        0        0        0        0        0        0   

                1/29/20  1/30/20  1/31/20  ...  4/9/20  4/10/20  4/11/20  \
Country/Region                             ...                             
Afghanistan           0        0        0  ...     484      521      555   
Albania               0        0        0  ...     409      416      433   
Algeria               0        0        0  ...    1666     1761     1825   
Andorra               0        0        0

KeyError: "['Country/Region'] not in index"

In [ ]:
import qgrid
qgrid_w = qgrid.show_grid(tmp_df, show_toolbar = True)
qgrid_w

# View the change in the world by using the slide bar

In [ ]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper,LogColorMapper, ColorBar, LogTicker
from bokeh.palettes import brewer, gray, Turbo256, inferno

from bokeh.io import curdoc
from bokeh.models import Slider, HoverTool, Column
from bokeh.layouts import widgetbox, row, column

#Input GeoJSON source that contains features for plotting.



In [ ]:
def json_data():
    merged = gdf.merge(country_df_code, on = 'country_code', how = 'left')
    
    merged.to_json()
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    return json_data


max_conf_bydate = 100




def json_bydate(dateval):
    #provide date in format m/dd/yy as str
    
    merged, max_conf_bydate = mergegdf_bydate(dateval)
    merged_json = json.loads(merged.to_json())
    
    json_data = json.dumps(merged_json)
    
    return json_data, max_conf_bydate


    

In [ ]:
palette = inferno(255)[::-1]
color_mapper = LogColorMapper(palette = palette, low = 1, high = max_confirmed, nan_color = '#d9d9d9')
color_bar = ColorBar(color_mapper=color_mapper, ticker = LogTicker(),  label_standoff=12, border_line_color=None, location=(0,0))
p = figure(title = f'Confirmed COVID-19 on {today}', plot_height = 500 , plot_width = 950, toolbar_location = None)
#Add patch renderer to figure. 
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None


In [ ]:
geosource = GeoJSONDataSource(geojson = json_data())
p.patches('xs','ys', source = geosource, fill_color = {'field' :'confirmed', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify figure layout.
p.add_layout(color_bar, 'right')

#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

In [ ]:
hover = HoverTool(tooltips = [ ('Country/region','@country'),('Covid', '')])

slider_def = date(2020, 3, 15)
dstr = get_strdate(slider_def)

p_slider = figure(title = f'Confirmed COVID-19 on {dstr}', plot_height = 500 , plot_width = 950)
p_slider.xgrid.grid_line_color = None
p_slider.ygrid.grid_line_color = None
#Add patch renderer to figure. 


geojs, maxconf = json_bydate(dstr)
geosource_date= GeoJSONDataSource(geojson = geojs)


color_mapper_date = LogColorMapper(palette = palette, low = 1, high = maxconf, nan_color = '#d9d9d9')

color_bar_date = ColorBar(color_mapper=color_mapper_date, ticker = LogTicker(),  label_standoff=12, 
                          border_line_color=None, location=(0,0))

p_slider.patches('xs','ys', source = geosource_date,
                 fill_color = {'field' :'confirmed', 'transform' : color_mapper_date},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)


p_slider.add_layout(color_bar_date, 'right')




In [ ]:
def update_plot(attr, old, new):
    
    dateval = new
    print(new, old, slider.value, dateval)
    p_slider.title.text = f'Confirmed COVID-19 on {dateval}'
    dstr = get_strdate(dateval)
    print(dstr, 'Geojson updated')
    
    new_data, maxconf = json_bydate(dstr)
    
    geosource_date.geojson = new_data
    color_mapper_date = LogColorMapper(palette = palette, low = 1, high = maxconf, nan_color = '#d9d9d9')

#     p_slider.patches('xs','ys', source = geosource_date,
#                      fill_color = {'field' :'confirmed', 'transform' : color_mapper_date},
#           line_color = 'black', line_width = 0.25, fill_alpha = 1)
    
#     layout = column(p_slider,widgetbox(slider))
    
#     show(layout)

   

    

In [ ]:
from bokeh.models.widgets import DateSlider
from bokeh.models import CustomJS


slider = DateSlider(title="Date Range: ", start=date(2020, 1, 22),
                    end=date.today(), value=slider_def)
layout = column(p_slider,widgetbox(slider))



slider.on_change('value', update_plot)


In [ ]:
    
curdoc().add_root(layout)
# curdoc().title = "DateRangeSlider Example"

#Display plot inline in Jupyter notebook
# output_notebook()
#Display plot
# curdoc().add_root(row(step_select, slider))

show(layout)

